<a href="https://colab.research.google.com/github/Fuenfgeld/ATeamDatenmanagementUndArchivierung/blob/main/IMECOS_master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kursarbeit Datenmanagement und -Archivierung WS 20/21
## Master-Code [IMECOS-Projekt](https://github.com/Fuenfgeld/ATeamDatenmanagementUndArchivierung)


---



## Initialisierung für Projekt-Administratoren

#### Datenbankinitialisierung & Quelldatenimport ([Wiki/Quelldatenmodell](https://github.com/Fuenfgeld/ATeamDatenmanagementUndArchivierung/wiki/Quelldatenmodell#datenbank-implementierung))

In [ ]:
# initialize the database, see import statements in *implementation.py
import requests
exec(requests.get('https://raw.githubusercontent.com/Fuenfgeld/ATeamDatenmanagementUndArchivierung/main/src/source_database_implementation.py').text)


*   Verbindung zu Googledrive herstellen (Ablageort der CSV-Dateien)
*   Verbindung zu SQLite herstellen
*   Datebankinitialisierung im *Memory*
*   [Quelldaten](https://github.com/Fuenfgeld/ATeamDatenmanagementUndArchivierung/wiki/Quelldatenmodell)import
*   Ausgabe der Connection- und Cursor-Syntax


---






### Datenqualität I - Datenexploration ([Wiki/Datenqualität](https://github.com/Fuenfgeld/ATeamDatenmanagementUndArchivierung/wiki/Datenqualit%C3%A4t#identifikation-von-redundanzen-und-unstimmigkeiten---data-clearing))
Die Folgenden beiden Zellen sind optional zur Ausführung der `data_clearing_SQL_queries`.

Legende zu den GitHub-Issues zur Datenqualität:


[#12](https://github.com/Fuenfgeld/ATeamDatenmanagementUndArchivierung/issues/12), 
[#13](https://github.com/Fuenfgeld/ATeamDatenmanagementUndArchivierung/issues/13),
[#14](https://github.com/Fuenfgeld/ATeamDatenmanagementUndArchivierung/issues/14),
[#15](https://github.com/Fuenfgeld/ATeamDatenmanagementUndArchivierung/issues/15),
[#30](https://github.com/Fuenfgeld/ATeamDatenmanagementUndArchivierung/issues/30).

In [ ]:
# decions about the design of the star scheme have followed several deeper looks into the data. Included here for documentation purpose.
# EXECUTION OF THIS CELL IS OPTIONAL!
# uncomment the following two lines after one another to see the documentation of this step.

# exec(requests.get('https://raw.githubusercontent.com/Fuenfgeld/ATeamDatenmanagementUndArchivierung/main/tools/data_clearing_SQL_queries.py').text)
# data_clearing_queries(30)


---





### ETL-Prozess ([Wiki/ETL-Prozess](https://github.com/Fuenfgeld/ATeamDatenmanagementUndArchivierung/wiki/Datenbereinigung-im-ETL-Prozess))

In [ ]:
# CREATE VIEW for central table  (but with DESCRIPTIONS)
cur.executescript(requests.get('https://raw.githubusercontent.com/Fuenfgeld/ATeamDatenmanagementUndArchivierung/main/tools/ETL_create_view_central.sql').text)
conn.commit()
# Perform Extraction, Transformation and Loading into new database
exec(requests.get('https://raw.githubusercontent.com/Fuenfgeld/ATeamDatenmanagementUndArchivierung/main/tools/ETL_process.py').text)


*   View auf Quelldatenbank erstellen ([ETL_create_view_central.sql](https://github.com/Fuenfgeld/ATeamDatenmanagementUndArchivierung/blob/main/tools/ETL_create_view_central.sql))
*   Datenextraktion ([ETL_process.py](https://github.com/Fuenfgeld/ATeamDatenmanagementUndArchivierung/blob/main/tools/ETL_process.py))
*   Verbindung zu SQLite herstellen
*   Datebankinitialisierung in öffentlichem Google Drive ([IMECOS_public/imecos_public.db](https://drive.google.com/file/d/1cnzmvXIE7CDIULh8hIHPCZJKEI4RKTiX/view?usp=sharing))
*   Anzeige der erstellten Post-ETL Tabellen inkl. Hash-Summen
*   Ausgabe der neuen Connection- und Cursor-Syntax


---


## Initialisierung für die Öffentlichkeit
Vor Ausführung ist eine Verlinkung der Datenbank in ein Google Drive notwendig.
### Siehe Hinweise unter [System-Umgebung](https://github.com/Fuenfgeld/ATeamDatenmanagementUndArchivierung/wiki/System-Umgebung) 

In [ ]:
import requests
exec(requests.get('https://raw.githubusercontent.com/Fuenfgeld/ATeamDatenmanagementUndArchivierung/main/src/public_datawarehouse_connection.py').text)


*   Verbindung zu SQLite herstellen
*   Datebank in öffentlichem Google Drive ([IMECOS_public/imecos_public.db](https://drive.google.com/file/d/1cnzmvXIE7CDIULh8hIHPCZJKEI4RKTiX/view?usp=sharing)) verbinden
*   Anzeige der Post-ETL Tabellen inkl. Hash-Summen:

Tabelle | Hash-Summe
----- | -----
encounters | -8064858177655843232
patients | -6782052882837843746
payers | 7404035236410964453
code_master | -6726061372865899500

*   Ausgabe der neuen Connection- und Cursor-Syntax


---


## Deskriptive Analysen des Forschungsdatensatzes

Import der [Seaborn](https://seaborn.pydata.org/)-Umgebungen zur grafischen Analyse & Befüllen der Dataframes:

In [ ]:
# import plot libraries
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# save tables from DB into dataframe
encounters = pd.read_sql_query("SELECT * FROM encounters", conn_new)
patients = pd.read_sql_query("SELECT * FROM patients", conn_new)

### Datenqualität II - Entfernung von Extremwerten ([Wiki/Datenqualität](https://github.com/Fuenfgeld/ATeamDatenmanagementUndArchivierung/wiki/Datenqualit%C3%A4t#identifikation-von-fehlern-und-ausrei%C3%9Fern-in-der-datenbasis---preliminary-data-analysis))

In [ ]:
# BOXPLOT of patients COV_EXP_RATIO
print("""patients.COV_EXP_RATO

""")
sns.boxplot(x=patients["COV_EXP_RATIO"])
plt.savefig('gdrive/Shareddrives/IMECOS/plots/COV_EXP_outlier.png')

In [ ]:
# remove outlier data from data warehouse via a SQL Delete Script
cur_new.executescript(requests.get('https://raw.githubusercontent.com/Fuenfgeld/ATeamDatenmanagementUndArchivierung/main/tools/data_quality_delete_outlier.sql').text)
conn_new.commit()

# update dataframes from DB
encounters = pd.read_sql_query("SELECT * FROM encounters", conn_new)
patients = pd.read_sql_query("SELECT * FROM patients", conn_new)

# verify successful deletion by again plotting BOXPLOT of patients COV_EXP_RATIO
print("""patients.COV_EXP_RATO nach Entfernung von Extremwerten

""")
sns.boxplot(x=patients["COV_EXP_RATIO"])



*   Deskriptive Statisktik der Dataframes



In [ ]:
# Descriptive statistics PATIENTS
print("Patients:")
patients.info()
print()
print("Encounters:")
encounters.info()



---
*   Berechung des Patientenalters & Altersverteilung in Studienpopulation:
